# Sweetviz

## Giới thiệu

- Sweetviz là thư viện Python mã nguồn mở dùng để Exploratory Data Analysis (EDA) tự động. Nó tạo ra báo cáo HTML trực quan, tương tác với các phân tích thống kê chi tiết.

## Các chức năng chính

-	Phân tích biến mục tiêu (Target analysis): Điều này cho thấy giá trị mục tiêu liên quan như thế nào đến các tính năng khác.
-	So sánh hai tập dữ liệu: Bạn có thể so sánh hai DataFrame (ví dụ: train và test) để xem phân bố biến, giá trị thiếu, mối quan hệ giữa các biến trong mỗi tập.
-	Liên kết giữa các loại biến hỗn hợp: Với dữ liệu mà có biến số (numerical), biến phân loại (categorical) hoặc hỗn hợp, SweetViz dùng các phép đo tương thích (ví dụ Pearson, hệ số không chắc chắn, correlation ratio) để đánh giá mối liên hệ.
-	Phát hiện kiểu dữ liệu tự động / ghi đè thủ công: SweetViz tự nhận biết biến số, phân loại, văn bản, nhưng bạn có thể ghi đè nếu muốn.
-	Báo cáo thống kê & tổng hợp: Cho mỗi biến: số lượng, giá trị thiếu, giá trị duy nhất, giá trị hay gặp nhất, min, max, trung vị, trung bình, độ lệch chuẩn, độ lệch (skewness), độ nhọn (kurtosis), hệ số biến động, v.v.


## Hướng dẫn sử dụng

- Yêu cầu: Phiên bản numpy 1.23.4, và python 3.9 để có độ tương thích cao với thưu viện này

- Import thư viện

In [ ]:
import pandas as pd

/home/vscode/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Tải dữ liệu từ file CSV
df = pd.read_csv('marketing_campaign.csv', delimiter='\t')

In [3]:
# Thông tin cơ bản về dữ liệu
print("Thông tin cơ bản về dataset:")
print(f"Kích thước: {df.shape}")
print(f"Số lượng missing values: {df.isnull().sum().sum()}")

Thông tin cơ bản về dataset:
Kích thước: (2240, 29)
Số lượng missing values: 24


- Bối cảnh
- Tuyên bố vấn đề

Phân tích Tính cách Khách hàng là một phân tích chi tiết về khách hàng lý tưởng của một công ty. Nó giúp doanh nghiệp hiểu rõ hơn về khách hàng của mình và giúp họ dễ dàng điều chỉnh sản phẩm theo nhu cầu, hành vi và mối quan tâm cụ thể của từng nhóm khách hàng khác nhau.

Phân tích tính cách khách hàng giúp doanh nghiệp điều chỉnh sản phẩm dựa trên khách hàng mục tiêu từ các phân khúc khách hàng khác nhau. Ví dụ, thay vì chi tiền để tiếp thị sản phẩm mới cho tất cả khách hàng trong cơ sở dữ liệu của công ty, doanh nghiệp có thể phân tích phân khúc khách hàng nào có khả năng mua sản phẩm cao nhất và sau đó chỉ tiếp thị sản phẩm cho phân khúc cụ thể đó.

- Mục đích nghiên cứu dataset này 
- Dataset này được thiết kế để phân tích:
    - Phân khúc khách hàng (Customer Segmentation)
    - Hiệu quả chiến dịch marketing (Campaign Response Analysis)
    - Hành vi mua hàng (Purchase Behavior)
    - Dự đoán phản hồi chiến dịch (Response Prediction)

In [ ]:
# Hiển thị thông tin chi tiết về các cột
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2240 entries, 0 to 2239
Data columns (total 29 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ID                   2240 non-null   int64  
 1   Year_Birth           2240 non-null   int64  
 2   Education            2240 non-null   object 
 3   Marital_Status       2240 non-null   object 
 4   Income               2216 non-null   float64
 5   Kidhome              2240 non-null   int64  
 6   Teenhome             2240 non-null   int64  
 7   Dt_Customer          2240 non-null   object 
 8   Recency              2240 non-null   int64  
 9   MntWines             2240 non-null   int64  
 10  MntFruits            2240 non-null   int64  
 11  MntMeatProducts      2240 non-null   int64  
 12  MntFishProducts      2240 non-null   int64  
 13  MntSweetProducts     2240 non-null   int64  
 14  MntGoldProds         2240 non-null   int64  
 15  NumDealsPurchases    2240 non-null   i

Ý nghĩa dataset
A. Các nhóm biến
1. Thông tin nhân khẩu học (People):
    - ID: Mã định danh khách hàng
    - Year_Birth: năm sinh khách hàng
    - Education: trình độ học vấn của khách hàng
    - Marital_Status: Tình trạng hôn nhân (8 loại bao gồm các giá trị bất thường như "YOLO", "Absurd", "Alone")
    - Income: Thu nhập hàng năm (có giá trị missing)
    - Kidhome: Số trẻ em nhỏ trong gia đình
    - Teenhome: Số thanh thiếu niên trong gia đình
    - Dt_customer: Ngày khách hàng đăng ký với công ty
    - Recency: Số ngày kể từ lần mua hàng cuối cùng của khách hàng
    - Complain: 1 nếu khách hàng đã khiếu nại trong 2 năm qua, 0 nếu không
2. Hành vi mua hàng (Products):
    - MntWines: Số tiền chi cho rượu vang trong 2 năm qua
    - MntFruits: Số tiền chi cho trái cây trong 2 năm qua
    - MntMeatProducts: Số tiền chi cho thịt trong 2 năm qua
    - MntFishProducts: Số tiền chi cho hải sản trong 2 năm qua
    - MntSweetProducts: Số tiền chi cho đồ ngọt trong 2 năm qua
    - MntGoldProds: Số tiền chi cho vàng trong 2 năm qua
3. Khuyến mãi (Promotion): 
    - NumDealsPurchases: Số lượng mua hàng được giảm giá
    - AcceptedCmp1: 1 if customer accepted the offer in the 1st campaign, 0 otherwise. 1 nếu khách hàng chấp nhận ưu đãi trong chiến dịch đầu tiên, 0 nếu không
    - AcceptedCmp2: 1 if customer accepted the offer in the 2nd campaign, 0 otherwise. Lần thứ 2
    - AcceptedCmp3: 1 if customer accepted the offer in the 3rd campaign, 0 otherwise. Lần thứ 3
    - AcceptedCmp4: 1 if customer accepted the offer in the 4th campaign, 0 otherwise. Lần thứ 4
    - AcceptedCmp5: 1 if customer accepted the offer in the 5th campaign, 0 otherwise. Lần thứ 5
    - Response: 1 if customer accepted the offer in the last campaign, 0 otherwise. Lần cuối
4. Địa điểm (Place): 
    - NumWebPurchases: Số lượng mua hàng được thực hiện thông qua trang web của công ty
    - NumCatalogPurchases: Số lần mua hàng qua Catalog
    - NumStorePurchases: Số lần mua hàng trực tiếp tại cửa hàng
    - NumWebVisitsMonth: Số lần truy cập web mỗi tháng


In [8]:
df.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
ID,2240.0,5592.159821,3246.662198,0.0,2828.25,5458.5,8427.75,11191.0
Year_Birth,2240.0,1968.805804,11.984069,1893.0,1959.00,1970.0,1977.00,1996.0
Income,2216.0,52247.251354,25173.076661,1730.0,35303.00,51381.5,68522.00,666666.0
Kidhome,2240.0,0.444196,0.538398,0.0,0.00,0.0,1.00,2.0
Teenhome,2240.0,0.506250,0.544538,0.0,0.00,0.0,1.00,2.0
Recency,2240.0,49.109375,28.962453,0.0,24.00,49.0,74.00,99.0
MntWines,2240.0,303.935714,336.597393,0.0,23.75,173.5,504.25,1493.0
MntFruits,2240.0,26.302232,39.773434,0.0,1.00,8.0,33.00,199.0
MntMeatProducts,2240.0,166.950000,225.715373,0.0,16.00,67.0,232.00,1725.0
MntFishProducts,2240.0,37.525446,54.628979,0.0,3.00,12.0,50.00,259.0


In [ ]:
df.isnull().sum()

ID                      0
Year_Birth              0
Education               0
Marital_Status          0
Income                 24
Kidhome                 0
Teenhome                0
Dt_Customer             0
Recency                 0
MntWines                0
MntFruits               0
MntMeatProducts         0
MntFishProducts         0
MntSweetProducts        0
MntGoldProds            0
NumDealsPurchases       0
NumWebPurchases         0
NumCatalogPurchases     0
NumStorePurchases       0
NumWebVisitsMonth       0
AcceptedCmp3            0
AcceptedCmp4            0
AcceptedCmp5            0
AcceptedCmp1            0
AcceptedCmp2            0
Complain                0
Z_CostContact           0
Z_Revenue               0
Response                0
dtype: int64

### Tạo báo cáo Phân tích đơn biến toàn bộ data

In [ ]:
# Khai báo thư viện sweetviz
import sweetviz as sv

In [ ]:
# Tạo báo cáo phân tích dữ liệu
report = sv.analyze(df)

Feature: ID                                  |▎         | [  3%]   00:00 -> (00:00 left)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:02 -> (00:00 left)


In [ ]:
# Hiển thị báo cáo dưới dạng file HTML
report.show_html("marketing_campaign.html")

### Tạo báo cáo với biến mục tiêu

In [ ]:
# Gán giá trị cho biến mục tiêu
data = df[["Response", "Income", "MntWines"]].copy()

In [7]:
# Tạo báo cáo SweetViz, với 'Response' là biến mục tiêu
report = sv.analyze([data, "Marketing Data"], target_feat="Response")

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


In [ ]:
# Hiển thị báo cáo dưới dạng file HTML
report.show_html("sweetviz_response_report.html")

### Tạo báo cáo so sánh Data set
- Trong SweetViz, ngoài sv.analyze() để xem toàn bộ dataset còn có thể:
    - sv.compare() → so sánh 2 tập dữ liệu.

In [4]:
# Chọn 2 biến cần phân tích
data1 = df[["Income", "MntWines"]].copy()

In [ ]:
# Giá trị trung vị của MntWines
median_wine = data1["MntWines"].median()

In [ ]:
# Tạo báo cáo so sánh 2 nhóm dựa trên giá trị trung vị của MntWines
report = sv.compare_intra(
    data1,
    data1["MntWines"] > median_wine,
    ["MntWines > median", "MntWines <= median"]
)

Feature: Income                              |███▎      | [ 33%]   00:00 -> (00:00 left)

Done! Use 'show' commands to display/save.   |██████████| [100%]   00:00 -> (00:00 left)


In [ ]:
# Hiển thị báo cáo dưới dạng file HTML
report.show_html("sweetviz_income_wine.html")

# Kết thúc